# FIXED: Field Mapping dengan Robust Column Handling
Mengatasi KeyError dengan safe column access dan column name normalization

In [ ]:
# Cell 1: Setup dengan Column Normalizer
import requests, hashlib, hmac, base64, json, time, os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pytz
import warnings
warnings.filterwarnings('ignore')

API_TOKEN = "aat.NTA.eyJ2IjoxLCJ1IjoxMDIyNDE2LCJkIjo5NDY3OTMsImFpIjo2MDMxMiwiYWsiOiIwOGRlZmNiMC1kNjEzLTQxYjgtOGI5YS0zOWNhNjQ1OWIzOTkiLCJhbiI6IkFwbGlrYXNpIC0gRGF0YSBDb2xsZWN0aW9uIiwiYXAiOiI2NzgwZTA1YS0wNjQ3LTQ2NzktYmEyYi1jMWE4YWEyZGZjYWUiLCJ0IjoxNzYwMDkwNzI4OTcwfQ.LemzKJp8Tgp+yacEUvUM8hgTrUbb2rhCgNrpW/WsznGtvusfjeVV7AkqPShw0QvqL4bUey3k7BbifqwJVtTVAFp84BfyrC0/YwM7Xl5zycmf95dsJZV8we1yD13KRDcG5PoBCqh5Y4CY0oz39gBPM5oMcy9PZixjYKSc8/LaqfMMZLfaYMPuGjb5ppq9KbLVqFWQSbheqRc=.McqkDx7gdPa9Fzn501K/Fsfzzb8N7iF08un74VZqQaA"
SIGNATURE_SECRET = "VdQuYB9APtdyJxgFOGr8CtSMUtjVjmeTxDRhnrnOuh9el8qft2h5RO61ftO1Zr5l"
wib = pytz.timezone('Asia/Jakarta')
end_date = datetime.now().strftime('%d/%m/%Y')
start_date = (datetime.now() - timedelta(days=90)).strftime('%d/%m/%Y')

# Helper: Find column dengan berbagai kemungkinan nama
def find_column(df, *possible_names):
    """Find first matching column from possible names (case insensitive)"""
    if df.empty: return None
    cols_lower = {c.lower(): c for c in df.columns}
    for name in possible_names:
        if name.lower() in cols_lower:
            return cols_lower[name.lower()]
    return None

def safe_get_col(df, *possible_names, default=''):
    """Safely get column value with multiple name options"""
    col = find_column(df, *possible_names)
    if col: return df[col]
    return pd.Series([default] * len(df))

def safe_float(val, default=0.0):
    try: return float(val) if val is not None and str(val).lower() not in ['nan','none',''] else default
    except: return default

def safe_str(val, default=''):
    return str(val) if val is not None and str(val).lower() not in ['nan','none',''] else default

print(f'📅 Period: {start_date} - {end_date}')
print('✅ Utilities loaded')

In [ ]:
# Cell 2: API Client
class AccurateAPI:
    def __init__(self, token, secret):
        self.token, self.secret, self.host = token, secret, None
        self.req_count, self.last_req = 0, time.time()
    def sign(self, ts): return base64.b64encode(hmac.new(self.secret.encode(), ts.encode(), hashlib.sha256).digest()).decode()
    def headers(self):
        ts = datetime.now(wib).strftime('%d/%m/%Y %H:%M:%S')
        return {'Authorization': f'Bearer {self.token}', 'X-Api-Timestamp': ts, 'X-Api-Signature': self.sign(ts), 'Content-Type': 'application/json'}
    def rate_limit(self):
        if time.time() - self.last_req >= 1: self.req_count, self.last_req = 0, time.time()
        if self.req_count >= 3: time.sleep(max(1.2 - (time.time() - self.last_req), 0)); self.req_count, self.last_req = 0, time.time()
        self.req_count += 1
    def init(self):
        r = requests.post('https://account.accurate.id/api/api-token.do', headers=self.headers())
        d = r.json(); k = 'database' if 'database' in d.get('d',{}) else 'data usaha'
        if d.get('s'): self.host = d['d'][k]['host']; print(f'✅ Connected: {self.host}'); return True
        return False
    def get(self, ep, p=None):
        if not self.host: return None
        for i in range(4):
            self.rate_limit()
            try:
                r = requests.get(f'{self.host}/accurate{ep}', headers=self.headers(), params=p, timeout=30)
                if r.status_code == 429: time.sleep(2**i + 5); continue
                return r.json()
            except: time.sleep(2**i)
        return None
    def pages(self, ep, p=None, mx=50):
        p = p or {}; data, pg = [], 1
        while pg <= mx:
            p['sp.page'], p['sp.pageSize'] = pg, 100
            r = self.get(ep, p)
            if not r or not r.get('s') or not r.get('d',[]): break
            data.extend(r['d'])
            if pg >= r.get('sp',{}).get('pageCount',1): break
            pg += 1; time.sleep(0.5)
        return data

client = AccurateAPI(API_TOKEN, SIGNATURE_SECRET)
client.init()

In [ ]:
# Cell 3: Pull Master Data & Show Columns
print('='*70)
print('📦 PULLING MASTER DATA')
print('='*70)

# Items - tanpa field filter untuk mendapat SEMUA kolom
print('\n📦 Items...')
df_items = pd.DataFrame(client.pages('/api/item/list.do', {}, 50))
print(f'   ✅ Items: {len(df_items)} records')
print(f'   📋 Available columns ({len(df_items.columns)}): {list(df_items.columns)}')

# Warehouses
print('\n🏭 Warehouses...')
df_wh = pd.DataFrame(client.pages('/api/warehouse/list.do', {}, 10))
print(f'   ✅ Warehouses: {len(df_wh)} records')
print(f'   📋 Columns: {list(df_wh.columns)}')

# Customers
print('\n👥 Customers...')
df_cust = pd.DataFrame(client.pages('/api/customer/list.do', {}, 30))
print(f'   ✅ Customers: {len(df_cust)} records')
print(f'   📋 Columns: {list(df_cust.columns)}')

# Vendors
print('\n🏪 Vendors...')
df_vend = pd.DataFrame(client.pages('/api/vendor/list.do', {}, 30))
print(f'   ✅ Vendors: {len(df_vend)} records')
print(f'   📋 Columns: {list(df_vend.columns)}')

In [ ]:
# Cell 4: Build Maps dengan Safe Column Access
print('='*70)
print('🗺️ BUILDING ENRICHMENT MAPS')
print('='*70)

# Product Map
prod_map = {}
prod_by_no = {}

# Detect column names
id_col = find_column(df_items, 'id', 'itemId')
no_col = find_column(df_items, 'no', 'itemNo', 'code')
name_col = find_column(df_items, 'name', 'itemName')
type_col = find_column(df_items, 'itemType', 'type')
cat_col = find_column(df_items, 'itemCategoryName', 'categoryName', 'category')
cost_col = find_column(df_items, 'avgCost', 'averageCost', 'cost')
price_col = find_column(df_items, 'unitPrice', 'sellingPrice', 'price')
unit_col = find_column(df_items, 'unit1Name', 'unitName', 'unit')

print(f'\n🔍 Detected Item Columns:')
print(f'   ID: {id_col}, No: {no_col}, Name: {name_col}')
print(f'   Type: {type_col}, Category: {cat_col}')
print(f'   Cost: {cost_col}, Price: {price_col}, Unit: {unit_col}')

for idx, row in df_items.iterrows():
    pid = row.get(id_col) if id_col else None
    if pid:
        data = {col: row.get(col) for col in df_items.columns}
        data['_id'] = pid
        data['_no'] = safe_str(row.get(no_col) if no_col else '')
        data['_name'] = safe_str(row.get(name_col) if name_col else '')
        data['_type'] = safe_str(row.get(type_col) if type_col else '')
        data['_category'] = safe_str(row.get(cat_col) if cat_col else '')
        data['_avgCost'] = safe_float(row.get(cost_col) if cost_col else 0)
        data['_unitPrice'] = safe_float(row.get(price_col) if price_col else 0)
        data['_unit1'] = safe_str(row.get(unit_col) if unit_col else '')
        prod_map[pid] = data
        if data['_no']:
            prod_by_no[data['_no']] = data

# Warehouse Map
wh_map = {}
wh_id_col = find_column(df_wh, 'id', 'warehouseId')
wh_name_col = find_column(df_wh, 'name', 'warehouseName')
wh_addr_col = find_column(df_wh, 'address', 'locationAddress')

for idx, row in df_wh.iterrows():
    wid = row.get(wh_id_col) if wh_id_col else None
    if wid:
        data = {col: row.get(col) for col in df_wh.columns}
        data['_id'] = wid
        data['_name'] = safe_str(row.get(wh_name_col) if wh_name_col else '')
        data['_address'] = safe_str(row.get(wh_addr_col) if wh_addr_col else '')
        wh_map[wid] = data

# Customer Map
cust_map = {}
cust_id_col = find_column(df_cust, 'id', 'customerId')
cust_name_col = find_column(df_cust, 'name', 'customerName')
cust_no_col = find_column(df_cust, 'customerNo', 'no', 'code')

for idx, row in df_cust.iterrows():
    cid = row.get(cust_id_col) if cust_id_col else None
    if cid:
        data = {col: row.get(col) for col in df_cust.columns}
        data['_id'] = cid
        data['_name'] = safe_str(row.get(cust_name_col) if cust_name_col else '')
        data['_no'] = safe_str(row.get(cust_no_col) if cust_no_col else '')
        cust_map[cid] = data

# Vendor Map
vend_map = {}
vend_id_col = find_column(df_vend, 'id', 'vendorId')
vend_name_col = find_column(df_vend, 'name', 'vendorName')
vend_no_col = find_column(df_vend, 'vendorNo', 'no', 'code')

for idx, row in df_vend.iterrows():
    vid = row.get(vend_id_col) if vend_id_col else None
    if vid:
        data = {col: row.get(col) for col in df_vend.columns}
        data['_id'] = vid
        data['_name'] = safe_str(row.get(vend_name_col) if vend_name_col else '')
        data['_no'] = safe_str(row.get(vend_no_col) if vend_no_col else '')
        vend_map[vid] = data

print(f'\n✅ Maps Built:')
print(f'   Products: {len(prod_map)} (by ID) + {len(prod_by_no)} (by No)')
print(f'   Warehouses: {len(wh_map)}')
print(f'   Customers: {len(cust_map)}')
print(f'   Vendors: {len(vend_map)}')

In [ ]:
# Cell 5: Enrichment Functions
def enrich_value(primary, *fallbacks):
    """Return first non-empty value"""
    if primary is not None and str(primary).lower() not in ['nan','none','','0','0.0']:
        return primary
    for fb in fallbacks:
        if fb is not None and str(fb).lower() not in ['nan','none','','0','0.0']:
            return fb
    return primary if primary is not None else (fallbacks[0] if fallbacks else '')

def enrich_numeric(primary, *fallbacks):
    """Return first non-zero numeric"""
    val = safe_float(primary)
    if val > 0: return val
    for fb in fallbacks:
        fval = safe_float(fb)
        if fval > 0: return fval
    return val

print('✅ Enrichment functions loaded')

In [ ]:
# Cell 6: Pull & Enrich Sales Details
print('='*70)
print('💰 SALES DETAILS with Full Enrichment')
print('='*70)

invs = client.pages('/api/sales-invoice/list.do', {'filter.transDate.>=': start_date, 'filter.transDate.<=': end_date}, 50)
print(f'   Invoices: {len(invs)} (processing 100)...')

sales_details = []
for i, inv in enumerate(invs[:100]):
    print(f'   {i+1}/100...', end='\r')
    r = client.get('/api/sales-invoice/detail.do', {'id': inv['id']})
    if r and r.get('s'):
        d = r['d']
        k = next((x for x in ['detailItem','items','detail'] if x in d and d[x]), None)
        
        cust = cust_map.get(inv.get('customerId'), {})
        
        if k:
            for it in d[k]:
                item_id = it.get('itemId', it.get('id'))
                prod = prod_map.get(item_id, prod_by_no.get(it.get('itemNo', it.get('no')), {}))
                wh = wh_map.get(it.get('warehouseId'), {})
                
                qty = enrich_numeric(it.get('quantity'), it.get('qty'))
                unit_price = enrich_numeric(it.get('unitPrice'), it.get('price'), prod.get('_unitPrice'))
                total_price = enrich_numeric(it.get('totalPrice'), it.get('amount'))
                if total_price == 0 and qty > 0 and unit_price > 0:
                    total_price = qty * unit_price
                
                sales_details.append({
                    'number': enrich_value(inv.get('number')),
                    'trans_date': enrich_value(inv.get('transDate')),
                    'item_no': enrich_value(it.get('itemNo'), prod.get('_no')),
                    'item_name': enrich_value(it.get('itemName'), prod.get('_name')),
                    'item_category': enrich_value(prod.get('_category')),
                    'quantity': qty,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'customer_name': enrich_value(inv.get('customerName'), cust.get('_name')),
                    'warehouse_name': enrich_value(it.get('warehouseName'), wh.get('_name')),
                })
    time.sleep(0.3)

df_sales = pd.DataFrame(sales_details)
print(f'\n✅ Sales: {len(df_sales)} records, {len(df_sales.columns)} columns')

In [ ]:
# Cell 7: Pull & Enrich Stock Mutations - FIXED
print('='*70)
print('🔄 STOCK MUTATIONS with Full Enrichment')
print('='*70)

# Filter inventory items menggunakan safe column access
type_col = find_column(df_items, 'itemType', 'type', 'productType')
if type_col:
    inv_items = df_items[df_items[type_col].isin(['INVENTORY','GROUP'])]
    print(f'   Found {len(inv_items)} inventory items (type column: {type_col})')
else:
    # Fallback: process all items
    inv_items = df_items
    print(f'   ⚠️ Type column not found, processing all {len(inv_items)} items')

print(f'   Processing first 100...')

mutations = []
for i, (idx, item) in enumerate(inv_items.head(100).iterrows()):
    print(f'   {i+1}/100...', end='\r')
    item_id = item.get(id_col) if id_col else item.get('id')
    r = client.get('/api/item/stock-mutation-history.do', {'id': item_id, 'startDate': start_date, 'endDate': end_date})
    if r and r.get('s'):
        prod = prod_map.get(item_id, {})
        avg_cost = enrich_numeric(prod.get('_avgCost'))
        
        for rec in r.get('d', []):
            wh = wh_map.get(rec.get('warehouseId'), {})
            increase = enrich_numeric(rec.get('quantityIn'), rec.get('increase'))
            decrease = enrich_numeric(rec.get('quantityOut'), rec.get('decrease'))
            
            value_in = enrich_numeric(rec.get('valueIn'))
            if value_in == 0 and increase > 0: value_in = increase * avg_cost
            
            value_out = enrich_numeric(rec.get('valueOut'))
            if value_out == 0 and decrease > 0: value_out = decrease * avg_cost
            
            mutations.append({
                'item_no': enrich_value(item.get(no_col) if no_col else '', prod.get('_no')),
                'item_name': enrich_value(item.get(name_col) if name_col else '', prod.get('_name')),
                'warehouse_name': enrich_value(rec.get('warehouseName'), wh.get('_name')),
                'trans_date': enrich_value(rec.get('transDate')),
                'transaction_type': enrich_value(rec.get('transactionTypeName'), rec.get('transactionType')),
                'increase': increase,
                'value_in': value_in,
                'decrease': decrease,
                'value_out': value_out,
                'balance': enrich_numeric(rec.get('balance')),
            })
    time.sleep(0.3)

df_mutations = pd.DataFrame(mutations)
print(f'\n✅ Mutations: {len(df_mutations)} records, {len(df_mutations.columns)} columns')

In [ ]:
# Cell 8: Current Stock
print('='*70)
print('📊 CURRENT STOCK')
print('='*70)

stks = client.pages('/api/item/list-stock.do', {}, 50)
print(f'   Stock records: {len(stks)}')

df_stock_raw = pd.DataFrame(stks)
print(f'   📋 Stock columns: {list(df_stock_raw.columns)}')

current_stock = []
for _, s in df_stock_raw.iterrows():
    item_id = s.get('id', s.get('itemId'))
    prod = prod_map.get(item_id, {})
    wh = wh_map.get(s.get('warehouseId'), {})
    
    on_stock = enrich_numeric(s.get('qtyStock'), s.get('stockAvailable'), s.get('quantity'))
    avg_cost = enrich_numeric(s.get('avgCost'), prod.get('_avgCost'))
    unit_price = enrich_numeric(s.get('unitPrice'), prod.get('_unitPrice'))
    
    current_stock.append({
        'item_no': enrich_value(s.get('no'), prod.get('_no')),
        'item_name': enrich_value(s.get('name'), prod.get('_name')),
        'item_category': enrich_value(s.get('itemCategoryName'), prod.get('_category')),
        'on_stock': on_stock,
        'warehouse_name': enrich_value(s.get('warehouseName'), wh.get('_name')),
        'avg_cost': avg_cost,
        'unit_price': unit_price,
        'stock_value': on_stock * avg_cost,
    })

df_stock = pd.DataFrame(current_stock)
print(f'✅ Stock: {len(df_stock)} records')

In [ ]:
# Cell 9: Export
print('='*70)
print('💾 EXPORTING')
print('='*70)

csv_folder = 'field_mapping_csv_output'
os.makedirs(csv_folder, exist_ok=True)

exports = [
    ('1_Sales_Details', df_sales),
    ('2_Stock_Mutations', df_mutations),
    ('3_Current_Stock', df_stock),
    ('4_Master_Items', df_items),
]

with pd.ExcelWriter('field_mapping_output_project1.xlsx', engine='openpyxl') as w:
    for name, df in exports:
        if not df.empty:
            # Quality check
            print(f'\n📊 {name} ({len(df)} rows, {len(df.columns)} cols)')
            for col in df.columns[:10]:  # First 10 cols
                null_pct = df[col].isna().sum() / len(df) * 100 if len(df) > 0 else 0
                if df[col].dtype in ['int64','float64']:
                    zero_pct = (df[col] == 0).sum() / len(df) * 100
                    status = '✅' if (null_pct < 10 and zero_pct < 40) else '⚠️'
                    print(f'   {status} {col}: null:{null_pct:.0f}% zero:{zero_pct:.0f}%')
                else:
                    status = '✅' if null_pct < 10 else '⚠️'
                    print(f'   {status} {col}: null:{null_pct:.0f}%')
            
            df.to_excel(w, sheet_name=name[:31], index=False)
            df.to_csv(f'{csv_folder}/{name}.csv', index=False, encoding='utf-8-sig')

print('\n🎉 Done!')